In [117]:
from binascii import hexlify, unhexlify
from string import printable
from random import randint
from Crypto.Cipher import AES

BLOCK = 16

In [118]:
def pad(s):
    return s + (BLOCK - len(s) % BLOCK) * chr(BLOCK - len(s) % BLOCK)

In [119]:
def randkey():
    return "".join([printable[randint(0, len(printable)-8)] for _ in range(BLOCK)]).encode()

randkey = randkey()

cipher = AES.new(randkey, AES.MODE_ECB)

In [120]:
password = "12345678901234567890123456789012"

padded = pad(password)

password = padded.encode()

cipher_text = cipher.encrypt(password)

password = hexlify(cipher_text).decode()
print(f"Password hex: {password}")

Password hex: 63e4a0a37ea3d664fe5734f1ba1da28268559f52d9e48704b1859b903369b0df00065f6ed6407506a1072b1441f8b77d


We can notice that trying to encrypt a password 17 chars long the lenght of the encrypted password changes. Also, we know that the pad function adds a whole block of padding only when the password + FLAG is longer than the block. This means that if our password is 16 chars then the message, password + FLAG, doesn't have any padding.

Running our own tests we can se that the length of the encrypted password has the same lenght of the case described before when the password + FLAG is 32 chars long.  This means that the FLAG is 32 - 16 = 16 chars long. 

This is the scenario for the ECB Oracle Attack!

In [121]:
from pwn import remote

r = remote('padding.challs.cyberchallenge.it', 9030)

flag = ""

for i in range(BLOCK - 1, -1, -1):
    to_send = "A" * i
    
    r.sendlineafter("Give me the password to encrypt:", to_send.encode())
    
    line = r.recvline().strip().decode()
    print(f"Iteration {BLOCK - i}, line: {line}")
    print(f"Flag: {flag}")
    
    colon_index = line.index(':')
    
    enc = line[colon_index + 2: colon_index + 2 + BLOCK * 2]

    r.recvline()
    
    for j in printable:
        # send string of len i + flag + j
        
        r.sendlineafter("Give me the password to encrypt:", (to_send + flag + j).encode())

        line = r.recvline().strip().decode()

        colon_index = line.index(':')

        compare = line[colon_index + 2: colon_index + 2 + BLOCK * 2]

        r.recvline()
        
        if compare == enc:
            flag += j
            break

print(f"Flag: {flag}")

r.close()

# flag: CCIT{r3m3mb3r_th

[x] Opening connection to padding.challs.cyberchallenge.it on port 9030
[x] Opening connection to padding.challs.cyberchallenge.it on port 9030: Trying 5.75.232.207
[+] Opening connection to padding.challs.cyberchallenge.it on port 9030: Done


/opt/homebrew/anaconda3/lib/python3.11/site-packages/pwnlib/tubes/tube.py:841: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  res = self.recvuntil(delim, timeout=timeout)


Iteration 1, line: Here is you secure encrypted password: ff6a306b866ffa4f104dc0b6534367046c7d5afe9b76c7ffa2cab1f25b8b2c06c8e7e1e8ab3bf45933728240d9615f20
Flag: 
Iteration 2, line: Here is you secure encrypted password: 39ce314463809c2de623fe1c19eead9ec2a9f58d898e5117cc29f6d3200c39cf3b4dc7fab9b3dbb8a8e22ca2cda79583
Flag: C
Iteration 3, line: Here is you secure encrypted password: 29ba3c2d9433dce6ca5441d00b7b01d3a4740f6934b3919e87e8d207bbaf1dd64a7b596607ac454c7d03e846db2db1a3
Flag: CC
Iteration 4, line: Here is you secure encrypted password: bf94c7862f58936f510aac6fa8ca1558d70ed27233d88b1d02ca513df8b373c8fc76397d46544a8a8601626c328ececc
Flag: CCI
Iteration 5, line: Here is you secure encrypted password: c75f184a4777991b03b1fa67e0a300f20de34c751ce80a7a545841a82e869aa7b136cb3f2d7b89032ec1b3b0e26a572e
Flag: CCIT
Iteration 6, line: Here is you secure encrypted password: b44566ec40b704078391f21d8721988787274cdee6e31e05190c510fd21c33ea0d4ab46d812eec7eb20753075a55030c
Flag: CCIT{
Iteration 7, 

Well, that's the flag... but it's not complete.
Doubling the block gives the full flag.

In [122]:
BLOCK = 32

# CCIT{r3m3mb3r_th3_3cb_p3ngu1n?}